# Compact version
Just the same as before, only now with less output

## Definitions


In [10]:
import openmesh as om
import numpy as np
import matplotlib.pyplot as plt

mesh = om.read_trimesh('T4.off')
V=mesh.points()
X=V[:,0]
Y=V[:,1]

def initActiveNodes():
    ActiveNodeList = []
    for vh in mesh.vertices():
        k = vh.idx()
        if V[k,0] == 0 or V[k,1] == 1:
            ActiveNodeList.append(vh)
    return ActiveNodeList

def initActiveFaceList():
    ActiveFaceList = []

    i=0
    for fh in mesh.faces():
        counter = 0
    
        for vh in mesh.fv(fh):
            if vh in ActiveNodeList:
                # print(vh.idx(),' in ', fh.idx())                
                counter = counter + 1
            
        if counter == 2:
            ActiveFaceList.append(fh)

    return ActiveFaceList

def init_TS_nodes():
    T = np.zeros(len(V))+7
    S = np.zeros(len(V))+7

    for vh in ActiveNodeList:    
        k = vh.idx()
        if V[k,0] == 0:
            T[k] = 0
            S[k] = 0
        elif V[k,1] == 1:
            T[k] = V[k,0]
            S[k] = 0
    
    return T, S

def init_TS_faces():
    Tf = np.zeros(len(mesh.faces()))+7
    Sf = np.zeros(len(mesh.faces()))+7

    for fh in ActiveFaceList:
        T_ABC = 0
        S_ABC = 0
        counter = 0
        for vh in mesh.fv(fh):
            if vh in ActiveNodeList:
                T_ABC = T_ABC + T[vh.idx()]
                S_ABC = S_ABC + S[vh.idx()]            
                counter = counter + 1
            
        if counter != 2:
            print('ERR at face', fh.idx())
        else:
            Tf[fh.idx()] = T_ABC / 2
            Sf[fh.idx()] = S_ABC / 2
            
    return Tf, Sf
    
# routine to get active face indices    
def getActiveFaceIndices():
    I_list = []
    for fh in ActiveFaceList:
        I_list.append(fh.idx())
        
    return I_list

    
def getActiveNodeIndices():
    I_list = []
    for fh in ActiveNodeList:
        I_list.append(fh.idx())

    return I_list


def getProcessedFaceIndices():
    I_list = []
    for fh in ProcessedFaceList:
        I_list.append(fh.idx())
        
    return I_list

def getObsoleteFaceIndices():
    I_list = []
    for fh in ObsoleteFaceList:
        I_list.append(fh.idx())
        
    return I_list

def get_T_C_dummy(T_A, T_B):
    return (T_A + T_B)/2 + 10

# ... update faces: don inline    



ActiveNodeList = initActiveNodes()
ActiveFaceList = initActiveFaceList()
    
T, S = init_TS_nodes()
Tf, Sf = init_TS_faces()

I_active_faces = getActiveFaceIndices()
print('active faces:', I_active_faces)


active faces: [1, 4, 5, 8, 11, 22, 40, 74, 75]


## Running

In [11]:
ProcessedFaceList = []
ObsoleteFaceList = []

ActiveNodeList = initActiveNodes()
ActiveFaceList = initActiveFaceList()

# print('len: list(I[K]) \t \t (id_A, id_B) -> id_C \n \n \n')
j=0
while len(ActiveFaceList)>0:
    j=j+1
    
    # (0) update the index
    I_active_faces = getActiveFaceIndices()

    I = np.array(I_active_faces)
    K = np.argsort(Tf[I], axis=0)

    # print('(len)', len(I_active_faces),': (f)', list(I[K]))
    
    # (1) obtain id of next face
    next_face_id = (I[K])[0]
    fh = mesh.face_handle(next_face_id)
    
    # quick check whether the face is obsolete,
    # which is a normal situation that should not be handled as an error
    # but by moving on to the next face

    counter = 0
    for vh in mesh.fv(fh):
        if vh in ActiveNodeList:
            counter = counter + 1    
    if counter == 3:
        # print('remove obsolete face:', fh.idx())
        ObsoleteFaceList.append(fh)
        ActiveFaceList.remove(fh)
        continue
            
    # (2) calculate value of outgoing vertex
    counter = 0
    for vh in mesh.fv(fh):
        if vh in ActiveNodeList:
            # print(vh.idx(),'ingoing')
            if counter == 0:
                counter = counter + 1
                id_A = vh.idx()
                T_A = T[vh.idx()]                
            elif counter == 1:
                counter = counter + 1
                id_B = vh.idx()
                T_B = T[vh.idx()]
            else:
                print('ERR: too many ingoing nodes')
                
        else:
            # print(vh.idx(),'outgoing')        
            id_C = vh.idx()
            vh_C = vh
            
    if counter == 2:
        T_C = get_T_C_dummy(T_A, T_B)
        T[id_C] = T_C
    else:
        print('ERR: missing inputs for calculating T_C')
        # T[id_C] = j
        
    # T[id_C] = j
    
    # (3) handle the list of active faces
    if (not vh_C in ActiveNodeList):
        ActiveNodeList.append(vh_C) # also ! 
    else:
        print('node already listed:', vh_C.idx())
    
    ActiveFaceList.remove(fh)
    ProcessedFaceList.append(fh)

    # (3b) agregate new faces to the list
    f_list = []
    print('(', id_A, ',', id_B, ') ->', id_C,'\t \t',len(I_active_faces),': ', list(I[K]))    
    for fvh in mesh.vf(vh_C):
        f_list.append(fvh.idx())
        # print('fv', fvh.idx())
        if (not fvh in ActiveFaceList) and (not fvh in ProcessedFaceList):
            counter = 0
            for vh in mesh.fv(fvh):
                if vh in ActiveNodeList:
                    counter = counter + 1
        
            if counter == 2:
                ActiveFaceList.append(fvh)
                # print('new face:', fvh.idx())
                
                counter = 0
                for vh in mesh.fv(fvh):
                    if vh in ActiveNodeList:
                        if counter == 0:
                            T_A = T[vh.idx()]
                            counter = counter + 1
                        elif counter == 1:
                            T_B = T[vh.idx()]
                    Tf[fvh.idx()] = (T_A + T_B)/2
                    Tf[fvh.idx()] = min(T_A, T_B)                    
            
            elif counter == 3:
                print('face duplicate registered:', fvh.idx())                
        
print('\n T: \n', T)
print('\n Tf: \n', Tf)

( 5 , 0 ) -> 30 	 	 9 :  [5, 8, 11, 74, 75, 1, 40, 22, 4]
( 5 , 10 ) -> 34 	 	 10 :  [8, 11, 74, 75, 0, 9, 1, 40, 22, 4]
( 15 , 10 ) -> 38 	 	 11 :  [11, 74, 75, 0, 9, 7, 1, 40, 22, 4, 33]
( 20 , 25 ) -> 46 	 	 12 :  [74, 75, 0, 9, 7, 43, 1, 40, 22, 4, 33, 6]
( 20 , 15 ) -> 42 	 	 13 :  [75, 0, 9, 7, 43, 76, 1, 35, 40, 22, 4, 33, 6]
( 30 , 0 ) -> 1 	 	 14 :  [0, 9, 7, 43, 76, 1, 35, 40, 22, 4, 33, 6, 44, 73]
( 46 , 26 ) -> 21 	 	 9 :  [35, 40, 22, 4, 33, 6, 44, 73, 32]
( 21 , 26 ) -> 47 	 	 10 :  [42, 40, 22, 4, 33, 6, 44, 73, 32, 72]
( 47 , 27 ) -> 22 	 	 10 :  [55, 22, 4, 33, 6, 44, 73, 32, 72, 41]
( 22 , 27 ) -> 48 	 	 11 :  [52, 22, 4, 33, 6, 44, 73, 32, 72, 41, 54]
( 48 , 28 ) -> 23 	 	 11 :  [57, 4, 33, 6, 44, 73, 32, 72, 41, 54, 51]
( 23 , 28 ) -> 49 	 	 12 :  [60, 4, 33, 6, 44, 73, 32, 72, 41, 54, 51, 56]
( 49 , 29 ) -> 24 	 	 12 :  [47, 33, 6, 44, 73, 32, 72, 41, 54, 51, 56, 59]
( 34 , 30 ) -> 6 	 	 12 :  [33, 6, 44, 73, 32, 72, 41, 54, 51, 56, 59, 48]
( 38 , 34 ) -> 11 	 	 13

## Checksum

In [12]:
I_active_nodes = getActiveNodeIndices()
print('active nodes:', I_active_nodes)


active nodes: [0, 5, 10, 15, 20, 25, 26, 27, 28, 29, 30, 34, 38, 46, 42, 1, 21, 47, 22, 48, 23, 49, 24, 6, 11, 16, 31, 2, 43, 35, 39, 44, 17, 45, 18, 19, 7, 12, 40, 32, 3, 41, 13, 14, 36, 37, 8, 33, 4, 9]


## Triangle solver
We have the index set $(a,b,c)$ or $(k_a, k_b, k_c)$ corresponding to the nodes $A, B, C$. It is expected that the calculation is from given values in $A, B$ to the values in $C$.

In terms of the two unknowns we have:

    given T_A, T_B, S_A, S_B
    get T_C, S_c
    
We have a system of two non-linear equations for two unknowns. Though an explict might be an option, the formulation is done implicitely.



We want to solve the two coupled equations:
\begin{align*}
    1. \qquad & \| \nabla T \| - f = 0 \\[2mm]
    2. \qquad & \frac{\nabla T}{\| \nabla T \|} = \nabla S
\end{align*}

There are various considerations:
1. For the first equation, a mayor issue is where (at which positions) to calculate $f$, at the barycenter of the triangle or at the outgoing point $C$? Actually, from an implementation point of view, both cases are equally possible. The evaluation at $C$ sounds as an implicit type formulations (similar to implicit methods to ODE), so this might be a choice.

2. The second equation is solved by integration. 
We denominate
\begin{align*}
    m = {\rm argmin} (S_a, S_b),
\end{align*}
sich that the constant $c_m$ for calculating
\begin{align*}
    S_c = x_c S_x + y_c S_y + c_m
\end{align*}
is taken from
\begin{align*}
    c_m = S_m - x_m S_x - y_m S_y,
\end{align*}
where $S_m, x_m, y_m$ are the coefficientes corresponding either to $S_a$ or $S_b$.
The gradient $(S_x, S_y)$ comes from the normalization
\begin{align*}
    (S_x, S_y) = \frac{(T_x, Y_y)}{\sqrt{T_x^2 + T_y^2}}
\end{align*}
Formally we have
\begin{align*}
    S_c - S_m = (x_c - x_m) S_x + (y_c - y_m) S_y
\end{align*}
i.e. anyway
\begin{align*}
    S_c - S_m - (x_c - x_m) S_x + (y_c - y_m) S_y = 0
\end{align*}


The two equations are coupled as there are the dependencies
\begin{align*}
    1. \qquad & T_a, T_b; \quad S_a, S_b, S_c \quad \rightarrow \quad T_C \\
    2. \qquad & T_a, T_b, T_c ; \quad S_a, S_b \quad \rightarrow \quad S_C
\end{align*}


The triangle solver is set up such that non-linear equations of the form $F(U) = 0$ need to be solved. This requires to count with initial guesses for the unknown values.



We want to experiment with the test situation
\begin{align*}
   (A, B) \rightarrow C : \qquad
    (P_0, P_5) \rightarrow P_{30}
\end{align*}
and try to assemble different components that contribute to a residual.


In [13]:
I = [0, 5, 30]
S = T.copy() 
print(V[I])

Ybar = sum(V[I,1])/3 # (sum(V[I])/3)[1]
Yc = V[I[2],1] # (V[I])[2,1]
print(Yc, Ybar)

Sbar=sum(S[I])/3
Sc=S[I[2]]
print(Sc, Sbar)

[[0.  0.  0. ]
 [0.  0.2 0. ]
 [0.5 0.1 0. ]]
0.10000000149011612 0.10000000149011612
10.0 3.3333333333333335


First of all we need to define a subroutine that calculates the gradient corresponding to a triangle.

With the required parameters need to be assambled we can calculate gradients.



In [14]:
def getGrad2(I, U):
    m = np.zeros((2, 2))
    b = np.zeros((2, 1))

    for k in range(2):
        m[k,0] = V[I[k+1],0]-V[I[0],0]
        m[k,1] = V[I[k+1],1]-V[I[0],1]
        b[k] = U[k+1]-U[0]
                
    g = np.linalg.solve(m, b)    
    c = np.linalg.cond(m)

    return c, g

U=np.zeros(3)
U[0]=T[I[0]]
U[1]=T[I[1]]
U[2]=max(T[I[0:2]])
# U=np.array([T[I[0:2]], max(T[I[0:2]])])

c, g=getGrad2(I, U)
print(g)
print('%.2f' % c)


[[0.]
 [0.]]
2.62


Now we are in the position to define the residual of the pair of coupled equations.

On the one hand the residual function called by the equation solver should depend on the unknown variables $u = (T_C, S_C)$, on the other hand the handling of the triangle information is done by the index set $I = (a,b,c)$. Therefore, a packing and unpacking procedure needs to be established.


* define a global index set, where the active index set is stored, to be accessed from the subroutine
* Though the parameters of the residual subroutine that is called by the equation solver are partially dummy parameters, as the global index set is the effectively used parameter set, the dummy parameter needs to be generated from the global index set.

some swap procedure...




In [20]:
def residual(u):

    I = global_I
    
    # swap procedure: the effectivaly used variables need to depend on the "partially dummy parameters"
    # such that a change of the parameters
    T[I[2]] = u[0]
    S[I[2]] = u[1]
    
    # for a better structure an extra subroutine is generated 
    r = get_res(I)
    
    return r


def get_res(I):
    U=np.zeros(3)
    U[0]=T[I[0]]
    U[1]=T[I[1]]
    # U[2]=max(T[I[0:2]])
    U[2]=T[I[2]]
    
    c, g=getGrad2(I, U)
    gnorm = np.linalg.norm(g)    
    
    
    Ybar=sum(V[I,1])/3 # (sum(V[I])/3)[1]
    Sbar=sum(S[I])/3
   
    Yc=V[I[2],1] # (V[I])[2,1]    
    Sc=S[I[2]]    
    
    rT = gnorm - Yc / (Sc + 1e-7)
    
    # proceed with the calculation of rS
    m=np.argmin(S[I[0:2]])
    Vm=V[I[m],0:2]
    Vc=V[I[2],0:2]
    
    # print(Vm) 
    gs = g / (gnorm+1e-9)
    cm = S[m]-np.dot(gs.transpose(), Vm)
    Sm = np.dot(gs.transpose(), Vc) + cm    
    rS = S[I[2]] - Sm
    
    r = [rT, rS[0]]
    
    return r

def cost(u):
    R=residual(u)
    c=np.inner(R,R)
    return c


global_I = [0, 5, 30]

u=np.zeros(2)
u[0]=T[global_I[0]]
u[1]=T[global_I[1]]
r = residual(u)
print(r)

r=get_res(I)
print(r)

[-1000000.0149011612, 0.0]
[-1000000.0149011612, 0.0]


Now we try to solve the residual equation as test case for the syntaxis. 


In [22]:
from scipy import optimize
from scipy.optimize import fsolve, newton_krylov

u=np.zeros(2)
I_global = [0, 5, 30]
u[0]=T[global_I[0]]
u[1]=T[global_I[1]]

res = optimize.broyden1(residual, u)
# res =  fsolve(residual, u)
# res = newton_krylov(residual, u, method='lgmres', verbose=1)


NoConvergence: [-2.29299241e+02  2.18450952e-04]

It seems that something does not work out.

## Explicit scheme: Triangle solver

The equation of the form
\begin{align*}
    \| \nabla T \| = f
\end{align*}
is discretized in a triangle $\Delta_{ABC}$ as
\begin{align*}
    H_C = f_C
\end{align*}
with the numerical Hamilton
\begin{align*}
    H_C = H(p_1, p_2), \qquad
    p_1 = \frac{T_C - T_A}{|C - A|}
    p_2 = \frac{T_C - T_B}{|C - B|}
\end{align*}
* $T_A, T_B, T_C$ are the solution values in the respective points
* $b = |C-A|, c = |B-A|, a = |C-B|$ are lengths

On an operative level, the numerical Hamiltonian is calculated as
\begin{align*}
    H_C = \begin{cases}
        \frac{1}{\sin \gamma} \sqrt{p_1^2 - 2 p_1 p_2 \cos \gamma + p_2^2} \quad & \text{if} \quad \star \\
        \min \{ T_C, T_A + b f_c, T_B + a f_c \} \quad & \text{otherwise}
    \end{cases}
\end{align*}
Here $\star$ means a set of fulfilled conditions:
\begin{align*}
    \text{Huygens principle}   \qquad & | T_B - T_A | \le |A-B| f_c \\
    \text{Evaluation ordering} \qquad & \theta < \beta \\
    \text{Causality}           \qquad & \theta + \alpha < \frac{\pi}{2}
\end{align*}
where
\begin{align*}
    \theta = \arcsin \Biggl( \frac{|T_B - T_A|}{|B-A| f_c} \Biggr)
\end{align*}
is an auxilliary threshold angle.

Given the information in two points, the information in the third point is calculated.

In the implementation, a crucial aspect is the handling of $f_C$, as the values are not know in a forward-moving process.
* iteration
* implicit calculation



In [8]:
def getAngle(v1, v2):
    angle = np.arccos(np.dot(v1, v2) / (np.linalg.norm(v1) * np.linalg.norm(v2)))    
    return angle


def getC(I):
    A = V[I[0],0:2]
    B = V[I[1],0:2]
    C = V[I[2],0:2]    
    TA = T[I[0]]
    TB = T[I[1]]
    

    a = sum((B-C)**2)**(1/2)
    b = sum((A-C)**2)**(1/2)
    c = sum((A-B)**2)**(1/2)
    
    # firstround = 1
    TC=min(TA,TB)
    TC=(TA+TB)/2
    
    p1 = abs(TC-TA)/b
    p2 = abs(TC-TB)/a
    p3 = abs(TA-TB)/c
    
    alpha = getAngle(B-A, C-A)
    beta = getAngle(A-B, C-B)
    gamma = getAngle(A-C, B-C)

    asum=alpha+beta+gamma
    if asum != np.pi:
        print('ERR: angles do not sum up to pi:', asum)
        
    YC = V[I[2],1] # C[1]
    SC = (TA+TB+1e-5)/2    
    fC = SC / (YC+1e-5)
    theta = np.arcsin(abs(TA-TB)/(c*fC))
    
    counter = 0
    if p2 < fC:
        counter = counter + 1
    else:
        print('ERR: Huygens principle')

    if theta < beta:
        counter = counter + 1
    else:
        print('ERR: Evaluation order')
        
    if theta + alpha < np.pi / 2:
        counter = counter + 1
    else:    
        print('ERR: Causality')        

    if counter == 3:
        HC = 1/np.sin(gamma)*np.sqrt(p1**2 - 2*p1*p2*np.cos(gamma)+p2**2)
    else:
        HC = min(TC, TA+b*fC, TB +a*fC)
    
    TC = HC
        
    return TC

I = [0, 5, 30]
TC = getC(I)
print(TC)

0.0


Revisting the calculation of the angles

In [9]:
I = [45, 19, 24]
I = [33, 3, 4]
TC = getC(I)

A = V[I[0],0:2]
B = V[I[1],0:2]
C = V[I[2],0:2]    
    
alpha = getAngle(B-A, C-A)
beta = getAngle(A-B, C-B)
gamma = getAngle(A-C, B-C)

print(alpha,beta,gamma)

ERR: Causality
2.746801528158816 0.19739556271548864 0.19739556271548864
